In [ ]:
# Install required packages for Google Colab
%pip install transformers sentence-transformers
%pip install supabase python-dotenv
%pip install sacremoses
%pip install fastapi uvicorn pydantic python-multipart
%pip install pyngrok
%pip install psutil  # For memory management
%pip install nest-asyncio  # Add this line

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress t

In [3]:
import os
import json
import logging
import traceback
from datetime import datetime
from typing import List, Dict, Any, Optional
import torch
from getpass import getpass

# AI Models
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoProcessor, AutoModelForImageTextToText

# Supabase
from supabase import create_client

# FastAPI
from fastapi import FastAPI, HTTPException, BackgroundTasks
from fastapi.middleware.cors import CORSMiddleware
from fastapi.responses import JSONResponse
import uvicorn

# Pydantic models for validation
from pydantic import BaseModel, Field

# Try to import Colab secrets (for Google Colab)
try:
    from google.colab import userdata
    COLAB_AVAILABLE = True
    print("✅ Google Colab environment detected")
except ImportError:
    # Create a mock userdata object for local development
    class MockUserdata:
        def get(self, key):
            return os.environ.get(key)
    
    userdata = MockUserdata()
    COLAB_AVAILABLE = False
    print("📱 Local environment detected - using environment variables")

# Setup logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

print("📦 All packages imported successfully!")
print(f"🕐 RAG session started at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"🔧 Using device: {'cuda' if torch.cuda.is_available() else 'cpu'}")
print(f"🌍 Environment: {'Google Colab' if COLAB_AVAILABLE else 'Local Development'}")

# Available medical models
MEDICAL_MODELS = {
    "1": {
        "name": "OpenBioLLM-8B",
        "path": "aaditya/OpenBioLLM-Llama3-8B",
        "description": "8B parameter medical LLM optimized for biomedical tasks",
        "type": "causal"
    },
    "2": {
        "name": "Med42-v2-8B",
        "path": "m42-health/Llama3-Med42-8B",
        "description": "8B parameter medical model from M42 Health based on Llama3",
        "type": "causal"
    }
}

print("\n🤖 Available Medical Models:")
for key, model in MEDICAL_MODELS.items():
    print(f"  {key}. {model['name']} - {model['description']}")

if COLAB_AVAILABLE:
    print("\nPlease select a model by entering the number (1 or 2):")
    print("Note: MedGemma-4B is available in a separate notebook (query_rag_medgemma.ipynb)")
else:
    print("\n💡 For local development:")
    print("   Set environment variables: SUPABASE_URL, SUPABASE_SERVICE_ROLE_KEY, NGROK_AUTH_TOKEN")
    print("   Or run this notebook in Google Colab for interactive setup")

📱 Local environment detected - using environment variables
📦 All packages imported successfully!
🕐 RAG session started at: 2025-07-18 16:57:13
🔧 Using device: cpu
🌍 Environment: Local Development

🤖 Available Medical Models:
  1. OpenBioLLM-8B - 8B parameter medical LLM optimized for biomedical tasks
  2. Med42-v2-8B - 8B parameter medical model from M42 Health based on Llama3

💡 For local development:
   Set environment variables: SUPABASE_URL, SUPABASE_SERVICE_ROLE_KEY, NGROK_AUTH_TOKEN
   Or run this notebook in Google Colab for interactive setup


In [4]:
# Modular Service Architecture
import asyncio
from concurrent.futures import ThreadPoolExecutor
from abc import ABC, abstractmethod

class BaseService(ABC):
    """Base class for all services"""
    
    def __init__(self):
        self.executor = ThreadPoolExecutor(max_workers=2)
    
    async def run_in_executor(self, func, *args):
        """Run CPU-intensive tasks in thread pool"""
        loop = asyncio.get_event_loop()
        return await loop.run_in_executor(self.executor, func, *args)

class QueryReformulatorService(BaseService):
    """Generates multiple query variations for RAG Fusion"""
    
    def __init__(self, openrouter_api_key: str = None):
        super().__init__()
        self.openrouter_api_key = openrouter_api_key
        self.base_url = "https://openrouter.ai/api/v1"
        self.model = "anthropic/claude-3-haiku"  # Fast and reliable
        
    async def reformulate_query(self, original_query: str, num_variations: int = 3) -> List[str]:
        """Generate multiple query variations using LLM"""
        try:
            # Simple prompt for query reformulation
            prompt = f"""You are a medical query enhancement specialist. Create {num_variations} different ways to ask this medical question for better information retrieval.

Original query: "{original_query}"

Generate {num_variations} variations that:
1. Use different medical terminology
2. Include synonyms and related terms
3. Focus on different aspects (symptoms, causes, treatments, etc.)
4. Are concise and clear

Return only the variations, one per line:"""

            if self.openrouter_api_key:
                # Use OpenRouter API
                variations = await self._call_openrouter_api(prompt, num_variations)
            else:
                # Fallback to simple rule-based reformulation
                variations = await self._rule_based_reformulation(original_query, num_variations)
            
            # Always include original query
            all_queries = [original_query] + variations
            return all_queries[:num_variations + 1]  # Limit total queries
            
        except Exception as e:
            print(f"❌ Query reformulation failed: {str(e)}")
            # Fallback to original query only
            return [original_query]
    
    async def _call_openrouter_api(self, prompt: str, num_variations: int) -> List[str]:
        """Call OpenRouter API for query reformulation"""
        try:
            headers = {
                "Authorization": f"Bearer {self.openrouter_api_key}",
                "Content-Type": "application/json"
            }
            
            payload = {
                "model": self.model,
                "messages": [{"role": "user", "content": prompt}],
                "max_tokens": 200,
                "temperature": 0.7
            }
            
            async with aiohttp.ClientSession() as session:
                async with session.post(
                    f"{self.base_url}/chat/completions",
                    headers=headers,
                    json=payload,
                    timeout=10
                ) as response:
                    if response.status == 200:
                        result = await response.json()
                        content = result['choices'][0]['message']['content']
                        
                        # Parse variations from response
                        variations = []
                        lines = content.strip().split('\n')
                        for line in lines:
                            line = line.strip()
                            if line and not line.startswith(('#', '-', '*', '1.', '2.', '3.')):
                                variations.append(line)
                        
                        return variations[:num_variations]
                    else:
                        raise Exception(f"API call failed: {response.status}")
                        
        except Exception as e:
            print(f"❌ OpenRouter API call failed: {str(e)}")
            raise
    
    async def _rule_based_reformulation(self, query: str, num_variations: int) -> List[str]:
        """Simple rule-based query reformulation as fallback"""
        variations = []
        
        # Medical synonyms mapping
        medical_synonyms = {
            'diabetes': ['diabetes mellitus', 'blood sugar', 'glucose', 'type 2 diabetes'],
            'hypertension': ['high blood pressure', 'elevated blood pressure', 'HTN'],
            'heart attack': ['myocardial infarction', 'MI', 'cardiac arrest', 'heart failure'],
            'stroke': ['cerebrovascular accident', 'CVA', 'brain attack'],
            'cancer': ['malignancy', 'tumor', 'neoplasm', 'carcinoma'],
            'pain': ['discomfort', 'ache', 'soreness', 'tenderness'],
            'fever': ['elevated temperature', 'pyrexia', 'hyperthermia'],
            'fatigue': ['tiredness', 'exhaustion', 'weakness', 'lethargy'],
            'nausea': ['sick to stomach', 'queasiness', 'upset stomach'],
            'headache': ['head pain', 'migraine', 'cephalalgia']
        }
        
        # Generate variations using synonyms
        for term, synonyms in medical_synonyms.items():
            if term.lower() in query.lower() and len(variations) < num_variations:
                for synonym in synonyms[:2]:  # Use max 2 synonyms per term
                    variation = query.lower().replace(term.lower(), synonym)
                    if variation != query.lower():
                        variations.append(variation.capitalize())
                        if len(variations) >= num_variations:
                            break
        
        # Add question format variations
        if len(variations) < num_variations:
            if '?' not in query:
                variations.append(f"{query}?")
            if 'what' not in query.lower():
                variations.append(f"What are {query.lower()}?")
            if 'how' not in query.lower():
                variations.append(f"How to {query.lower()}?")
        
        return variations[:num_variations]

class EmbedderService(BaseService):
    """Handles text embedding generation on CPU"""
    
    def __init__(self):
        super().__init__()
        self.model = None
        self.device = 'cpu'
    
    async def initialize(self):
        """Initialize embedding model on CPU"""
        if self.model is None:
            from sentence_transformers import SentenceTransformer
            self.model = SentenceTransformer('NeuML/pubmedbert-base-embeddings', device='cpu')
            print("✅ EmbedderService: Model loaded on CPU")
    
    async def embed_text(self, text: str) -> List[float]:
        """Generate embeddings for text"""
        await self.initialize()
        
        # Run embedding on CPU thread pool
        embedding = await self.run_in_executor(self.model.encode, [text])
        return embedding[0].tolist()
    
    async def embed_batch(self, texts: List[str]) -> List[List[float]]:
        """Generate embeddings for multiple texts"""
        await self.initialize()
        
        # Run batch embedding on CPU thread pool
        embeddings = await self.run_in_executor(self.model.encode, texts)
        return embeddings.tolist()

class RetrieverService(BaseService):
    """Handles document retrieval from Supabase"""
    
    def __init__(self, supabase_client, embedder_service: EmbedderService):
        super().__init__()
        self.supabase = supabase_client
        self.embedder = embedder_service
        self.top_k = 5
        self.similarity_threshold = 0.5
    
    async def retrieve_documents(self, query: str, top_k: int = None) -> List[dict]:
        """Retrieve relevant documents using vector search"""
        top_k = top_k or self.top_k
        
        # Generate query embedding
        query_embedding = await self.embedder.embed_text(query)
        
        # Search Supabase
        try:
            result = self.supabase.rpc('search_embeddings', {
                'query_embedding': query_embedding,
                'match_threshold': self.similarity_threshold,
                'match_count': top_k
            }).execute()
            
            if result.data:
                documents = []
                for i, doc in enumerate(result.data):
                    documents.append({
                        'content': doc.get('chunk_content', ''),
                        'similarity_score': doc.get('similarity', 0.0),
                        'metadata': {
                            'title': doc.get('title', 'Medical Document'),
                            'source': doc.get('source', 'unknown'),
                            'topic': doc.get('topic', 'general'),
                            'document_type': doc.get('document_type', 'unknown'),
                            'document_id': doc.get('document_id', '')
                        },
                        'rank': i + 1,
                        'doc_id': doc.get('document_id', '')
                    })
                return documents
            else:
                return []
                
        except Exception as e:
            print(f"❌ RetrieverService error: {str(e)}")
            return []

class GeneratorService(BaseService):
    """Handles text generation using medical LLMs on GPU"""
    
    def __init__(self, model_name: str, tokenizer, medical_model):
        super().__init__()
        self.model_name = model_name
        self.tokenizer = tokenizer
        self.medical_model = medical_model
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    async def generate_response(self, prompt: str, max_tokens: int = 150) -> str:
        """Generate medical response using the selected model"""
        try:
            # Run generation on GPU thread pool
            response = await self.run_in_executor(self._generate_sync, prompt, max_tokens)
            return response
        except Exception as e:
            print(f"❌ GeneratorService error: {str(e)}")
            return f"I apologize, but I encountered an error: {str(e)}"
    
    def _generate_sync(self, prompt: str, max_tokens: int) -> str:
        """Synchronous generation method for thread pool"""
        inputs = self.tokenizer(prompt, return_tensors="pt", truncation=True, max_length=2048)
        if torch.cuda.is_available():
            inputs = {k: v.to("cuda") for k, v in inputs.items()}
        
        input_len = inputs['input_ids'].shape[1]
        
        generation_params = {
            "max_new_tokens": max_tokens,
            "temperature": 0.7,
            "do_sample": True,
            "repetition_penalty": 1.1,
            "top_p": 0.9
        }
        
        if hasattr(self.tokenizer, 'pad_token_id') and self.tokenizer.pad_token_id is not None:
            generation_params["pad_token_id"] = self.tokenizer.pad_token_id
        if hasattr(self.tokenizer, 'eos_token_id') and self.tokenizer.eos_token_id is not None:
            generation_params["eos_token_id"] = self.tokenizer.eos_token_id
        
        with torch.no_grad():
            outputs = self.medical_model.generate(**inputs, **generation_params)
        
        response = self.tokenizer.decode(outputs[0][input_len:], skip_special_tokens=True)
        return response.strip()

print("✅ All service classes defined!")
print("   📍 QueryReformulatorService: Multi-query generation")
print("   📍 EmbedderService: Text embeddings on CPU")
print("   📍 RetrieverService: Document retrieval from Supabase")
print("   📍 GeneratorService: Medical text generation")

✅ All service classes defined!
   📍 QueryReformulatorService: Multi-query generation
   📍 EmbedderService: Text embeddings on CPU
   📍 RetrieverService: Document retrieval from Supabase
   📍 GeneratorService: Medical text generation


In [ ]:
# Setup Supabase connection
print("🗄️ Setting up Supabase connection...")

if COLAB_AVAILABLE:
    print("📋 Required Colab secrets:")
    print("   1. SUPABASE_URL - Your project URL (e.g., https://abc123.supabase.co)")
    print("   2. SUPABASE_SERVICE_ROLE_KEY - Your service role key (NOT anon key)")
    print("   3. NGROK_AUTH_TOKEN - Your ngrok authentication token")
    print("")
    print("🔑 To set these secrets:")
    print("   1. Click the 🔑 key icon in the left sidebar")
    print("   2. Add the three secrets listed above")
    print("   3. Re-run this cell")
    print("")
else:
    print("💻 For local development, set environment variables:")
    print("   export SUPABASE_URL='https://your-project.supabase.co'")
    print("   export SUPABASE_SERVICE_ROLE_KEY='your-service-role-key'")
    print("   export NGROK_AUTH_TOKEN='your-ngrok-token'")
    print("")

try:
    supabase_url = userdata.get('SUPABASE_URL')
    supabase_key = userdata.get('SUPABASE_SERVICE_ROLE_KEY')
    ngrok_token = userdata.get('NGROK_AUTH_TOKEN')
    
    if COLAB_AVAILABLE:
        print("✅ Successfully retrieved secrets from Colab")
    else:
        print("✅ Successfully retrieved environment variables")
    
except Exception as e:
    print(f"❌ Error retrieving credentials: {str(e)}")
    if COLAB_AVAILABLE:
        print("🔧 Make sure you've added the required secrets in Colab:")
        print("   • SUPABASE_URL")
        print("   • SUPABASE_SERVICE_ROLE_KEY") 
        print("   • NGROK_AUTH_TOKEN")
    else:
        print("🔧 Make sure you've set the required environment variables:")
        print("   • SUPABASE_URL")
        print("   • SUPABASE_SERVICE_ROLE_KEY") 
        print("   • NGROK_AUTH_TOKEN")
    raise

# Validate the inputs
if not supabase_url or not supabase_key:
    raise ValueError("❌ Both Supabase URL and Service Role Key are required!")

if not supabase_url.startswith('https://'):
    raise ValueError("❌ Supabase URL should start with 'https://'")

if not supabase_key.startswith('eyJ'):
    print("⚠️ WARNING: Service role keys typically start with 'eyJ'")
    print("   You might be using the anon key instead of service_role key")
    
if len(supabase_key) < 100:
    print("⚠️ WARNING: Service role keys are typically very long (200+ characters)")
    print("   You might be using the anon key instead of service_role key")

try:
    supabase = create_client(supabase_url, supabase_key)
    print("✅ Supabase client initialized")
except Exception as e:
    print(f"❌ Failed to initialize Supabase client: {str(e)}")
    print("🔧 Common issues:")
    print("   • Wrong API key type (use service_role, not anon)")
    print("   • Typo in URL or key")
    print("   • Key might be expired or regenerated")
    raise

# Configuration
CONFIG = {
    "top_k": 5,
    "similarity_threshold": 0.5,
    "max_context_length": 2000,
    "max_response_length": 150,
}

print(f"\n⚙️ RAG Configuration:")
print(f"   🎯 Retrieve top {CONFIG['top_k']} similar documents")
print(f"   📊 Similarity threshold: {CONFIG['similarity_threshold']}")
print(f"   📏 Max context length: {CONFIG['max_context_length']} chars")

In [ ]:
# Pydantic models for request/response validation
class AskRequest(BaseModel):
    question: str = Field(..., min_length=1, max_length=1000, description="The medical question to ask")
    session_id: Optional[str] = Field(default="default", description="Chat session identifier")
    history: Optional[List[dict]] = Field(default=[], description="Previous chat history")

class AskResponse(BaseModel):
    response: str = Field(..., description="AI generated response")
    sources: List[dict] = Field(..., description="Source documents used")
    chat_history: List[dict] = Field(..., description="Updated chat history")
    session_id: str = Field(..., description="Session identifier")
    mockMode: bool = Field(default=False, description="Whether response is from mock mode")
    metadata: dict = Field(..., description="Response metadata")

class EmbedRequest(BaseModel):
    text: str = Field(..., min_length=1, max_length=5000, description="Text to embed")

class EmbedResponse(BaseModel):
    embedding: List[float] = Field(..., description="Generated embedding vector")
    dimensions: int = Field(..., description="Embedding dimensions")
    model: str = Field(..., description="Model used for embedding")
    device: str = Field(..., description="Device used for embedding")

class GenerateRequest(BaseModel):
    query: str = Field(..., min_length=1, max_length=1000, description="Query text")
    context: str = Field(default="", description="Context information")
    history: List[dict] = Field(default=[], description="Chat history")
    max_tokens: int = Field(default=200, ge=1, le=1000, description="Maximum tokens to generate")
    temperature: float = Field(default=0.7, ge=0.0, le=2.0, description="Generation temperature")

class GenerateResponse(BaseModel):
    answer: str = Field(..., description="Generated answer")
    model: str = Field(..., description="Model used for generation")
    model_path: str = Field(..., description="Model path")
    context_used: bool = Field(..., description="Whether context was used")
    history_used: bool = Field(..., description="Whether history was used")

class QueryRequest(BaseModel):
    query: str = Field(..., min_length=1, max_length=1000, description="Search query")
    top_k: int = Field(default=5, ge=1, le=20, description="Number of documents to retrieve")

class QueryResponse(BaseModel):
    documents: List[dict] = Field(..., description="Retrieved documents")
    total_found: int = Field(..., description="Total documents found")
    query: str = Field(..., description="Original query")

class RetrieveRequest(BaseModel):
    query: str = Field(..., min_length=1, max_length=1000, description="Search query for fusion retrieval")
    top_k: Optional[int] = Field(default=5, ge=1, le=20, description="Number of documents to retrieve")

class RetrieveResponse(BaseModel):
    query: str = Field(..., description="Original query")
    documents: List[dict] = Field(..., description="Retrieved documents with fusion metadata")
    total_found: int = Field(..., description="Total documents found")
    fusion_metadata: dict = Field(..., description="RAG Fusion specific metadata")

class HealthResponse(BaseModel):
    status: str = Field(..., description="System status")
    model: str = Field(..., description="Active model name")
    model_path: str = Field(..., description="Model path")
    embedding_device: str = Field(..., description="Embedding device")
    database: str = Field(..., description="Database type")
    documents_in_db: int = Field(..., description="Number of documents in database")
    embeddings_in_db: int = Field(..., description="Number of embeddings in database")
    rag_system: str = Field(..., description="RAG system type")
    chat_support: bool = Field(..., description="Chat support status")
    active_sessions: int = Field(..., description="Number of active sessions")
    gpu_memory: dict = Field(..., description="GPU memory information")

print("✅ Pydantic models defined for request/response validation")
print("   📍 Added RetrieveRequest/Response for RAG Fusion endpoints")
print("   📍 All endpoints have proper validation and documentation")

In [ ]:
# Memory Management Utilities
import gc
import psutil

def get_gpu_memory_info():
    """Get detailed GPU memory information"""
    if not torch.cuda.is_available():
        return "No GPU available"
    
    device = torch.cuda.current_device()
    allocated = torch.cuda.memory_allocated(device) / 1024**3
    reserved = torch.cuda.memory_reserved(device) / 1024**3
    total = torch.cuda.get_device_properties(device).total_memory / 1024**3
    free = total - allocated
    
    return {
        "allocated": f"{allocated:.2f} GB",
        "reserved": f"{reserved:.2f} GB", 
        "total": f"{total:.2f} GB",
        "free": f"{free:.2f} GB",
        "percentage_used": f"{(allocated/total)*100:.1f}%"
    }

def clear_gpu_memory():
    """Clear GPU memory cache"""
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        gc.collect()
        print("🧹 GPU cache cleared")
    else:
        print("📱 No GPU to clear")

# Show current GPU memory status
print("🔍 Current GPU Memory Status:")
memory_info = get_gpu_memory_info()
if isinstance(memory_info, dict):
    for key, value in memory_info.items():
        print(f"   {key}: {value}")
else:
    print(f"   {memory_info}")

print("\n✅ Memory management utilities ready!")
print("   Use get_gpu_memory_info() to check memory")
print("   Use clear_gpu_memory() to free cache")

In [ ]:
# Model selection and loading
print("🤖 Select your medical model:")

# For interactive environments (Colab), ask for input
# For automated environments, default to model 1
if COLAB_AVAILABLE:
    model_choice = input("Enter model number (1 or 2): ").strip()
else:
    # Default to model 1 for local development, but allow override via environment
    model_choice = os.environ.get('MODEL_CHOICE', '1')
    print(f"Using model choice: {model_choice} (set MODEL_CHOICE env var to override)")

if model_choice not in MEDICAL_MODELS:
    print(f"❌ Invalid choice '{model_choice}'. Defaulting to OpenBioLLM-8B (option 1)")
    model_choice = "1"

selected_model = MEDICAL_MODELS[model_choice]
model_name = selected_model["name"]
model_path = selected_model["path"]
model_type = selected_model["type"]

print(f"🧠 Loading {model_name} for medical text generation...")
print(f"📦 Model path: {model_path}")
print(f"🔧 Model type: {model_type}")

# Check GPU memory before loading
if torch.cuda.is_available():
    print(f"🔍 GPU Memory before loading: {torch.cuda.memory_allocated()/1024**3:.2f} GB allocated, {torch.cuda.memory_reserved()/1024**3:.2f} GB reserved")

try:
    # Standard causal LM handling (models 1 and 2 are both causal models)
    print("🔤 Loading tokenizer...")
    tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
    
    print("🧠 Loading model (this may take a few minutes)...")
    medical_model = AutoModelForCausalLM.from_pretrained(
        model_path,
        torch_dtype=torch.float16,
        low_cpu_mem_usage=True,
        device_map="auto",
        trust_remote_code=True
    )
    
    # Set device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    # Set pad token if not available
    if hasattr(tokenizer, 'pad_token') and tokenizer.pad_token is None:
        if tokenizer.eos_token is not None:
            tokenizer.pad_token = tokenizer.eos_token
        else:
            tokenizer.pad_token = tokenizer.unk_token
    
    # Store model info globally
    MODEL_INFO = {
        "name": model_name,
        "path": model_path,
        "choice": model_choice,
        "type": model_type
    }
    
    # Check GPU memory after loading
    if torch.cuda.is_available():
        print(f"🔍 GPU Memory after loading: {torch.cuda.memory_allocated()/1024**3:.2f} GB allocated, {torch.cuda.memory_reserved()/1024**3:.2f} GB reserved")
    
    print(f"✅ {model_name} loaded and ready on {device}")
    print(f"🎯 Selected model: {model_name}")
    
except Exception as e:
    print(f"❌ Error loading {model_name}: {str(e)}")
    print("🔄 Falling back to a smaller model...")
    
    # Fallback to a more reliable model
    fallback_path = "microsoft/DialoGPT-medium"
    print(f"🔄 Loading fallback model: {fallback_path}")
    
    tokenizer = AutoTokenizer.from_pretrained(fallback_path)
    medical_model = AutoModelForCausalLM.from_pretrained(
        fallback_path,
        torch_dtype=torch.float16,
        low_cpu_mem_usage=True,
        device_map="auto"
    )
    
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
    
    MODEL_INFO = {
        "name": "DialoGPT-medium (Fallback)",
        "path": fallback_path,
        "choice": "fallback"
    }
    
    print(f"✅ Fallback model loaded successfully")
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Pre-load the embedding model to avoid CUDA errors during Flask requests
print("🔍 Pre-loading embedding model to avoid CUDA memory conflicts...")
try:
    from sentence_transformers import SentenceTransformer
    
    # Check if there's enough GPU memory for the embedding model
    if torch.cuda.is_available():
        total_memory = torch.cuda.get_device_properties(0).total_memory
        allocated_memory = torch.cuda.memory_allocated()
        free_memory = total_memory - allocated_memory
        
        print(f"🔍 Available GPU memory: {free_memory/1024**3:.2f} GB")
        
        # If less than 2GB free, use CPU for embeddings
        if free_memory < 2 * 1024**3:  # 2GB threshold
            print("⚠️ Limited GPU memory - using CPU for embedding model")
            embedding_model = SentenceTransformer('NeuML/pubmedbert-base-embeddings', device='cpu')
            EMBEDDING_DEVICE = 'cpu'
        else:
            print("✅ Sufficient GPU memory - using GPU for embedding model")
            embedding_model = SentenceTransformer('NeuML/pubmedbert-base-embeddings')
            EMBEDDING_DEVICE = 'cuda'
    else:
        print("📱 No GPU available - using CPU for embedding model")
        embedding_model = SentenceTransformer('NeuML/pubmedbert-base-embeddings', device='cpu')
        EMBEDDING_DEVICE = 'cpu'
    
    print(f"✅ Embedding model loaded on {EMBEDDING_DEVICE}")
    
except Exception as e:
    print(f"⚠️ Error pre-loading embedding model: {str(e)}")
    print("🔄 Will load embedding model on-demand with CPU fallback")
    embedding_model = None
    EMBEDDING_DEVICE = 'cpu'

In [ ]:
# Initialize RAG Fusion Services
print("🔧 Initializing RAG Fusion services...")

# Initialize base services
embedder_service = EmbedderService()
base_retriever_service = RetrieverService(supabase, embedder_service)
generator_service = GeneratorService(MODEL_INFO['name'], tokenizer, medical_model)

# Initialize RAG Fusion components
query_reformulator = QueryReformulatorService()  # No API key for now - uses rule-based fallback
fusion_retriever = FusionRetrieverService(base_retriever_service, query_reformulator)

# Create enhanced RAG controller with fusion support
rag_controller = RAGController(
    embedder=embedder_service,
    retriever=base_retriever_service,
    generator=generator_service,
    fusion_retriever=fusion_retriever
)

print("✅ RAG Fusion services initialized!")
print(f"   📍 QueryReformulatorService: Multi-query generation")
print(f"   📍 FusionRetrieverService: RRF-based document fusion")
print(f"   📍 RAGController: Complete pipeline with fusion support")
print(f"   📍 Enhanced services ready for {MODEL_INFO['name']}")

# RAG Fusion Configuration
FUSION_CONFIG = {
    "enabled": True,
    "max_query_variations": 4,
    "rrf_k_parameter": 60,
    "retrieval_multiplier": 2,  # Get 2x docs for fusion
    "use_openrouter": False,  # Set to True if you have API key
    "openrouter_api_key": None,  # Add your key here
    "fallback_to_single": True  # Fallback if fusion fails
}

print("\n⚙️ RAG Fusion Configuration:")
for key, value in FUSION_CONFIG.items():
    print(f"   {key}: {value}")

# Update services with configuration
if FUSION_CONFIG["use_openrouter"] and FUSION_CONFIG["openrouter_api_key"]:
    query_reformulator.openrouter_api_key = FUSION_CONFIG["openrouter_api_key"]
    print("🔑 OpenRouter API configured for query reformulation")
else:
    print("📝 Using rule-based query reformulation (fallback mode)")

In [ ]:
# RAG Fusion: Fusion Retriever Service
from collections import defaultdict
import math

class FusionRetrieverService(BaseService):
    """Combines multiple query retrievals using Reciprocal Rank Fusion (RRF)"""
    
    def __init__(self, base_retriever: RetrieverService, reformulator: QueryReformulatorService):
        super().__init__()
        self.base_retriever = base_retriever
        self.reformulator = reformulator
        self.k = 60  # RRF parameter (typically 60)
        self.max_queries = 4  # Maximum number of query variations
        
    async def retrieve_documents_fusion(self, query: str, top_k: int = 5) -> List[dict]:
        """Retrieve documents using RAG Fusion with multiple query variations"""
        try:
            print(f"🔄 RAG Fusion: Generating query variations...")
            
            # Step 1: Generate query variations
            query_variations = await self.reformulator.reformulate_query(query, self.max_queries - 1)
            print(f"📝 Generated {len(query_variations)} query variations")
            
            # Step 2: Retrieve documents for each variation
            all_documents = []
            for i, variation in enumerate(query_variations):
                print(f"🔍 Retrieving for variation {i+1}: '{variation[:50]}...'")
                docs = await self.base_retriever.retrieve_documents(variation, top_k * 2)  # Get more docs for fusion
                
                # Add query source info
                for doc in docs:
                    doc['query_source'] = i
                    doc['query_variation'] = variation
                
                all_documents.extend(docs)
            
            # Step 3: Apply Reciprocal Rank Fusion
            print(f"🔗 Fusing {len(all_documents)} documents from {len(query_variations)} queries...")
            fused_docs = await self._apply_rrf(all_documents, top_k)
            
            # Step 4: Deduplicate and format results
            final_docs = await self._deduplicate_and_format(fused_docs, top_k)
            
            print(f"✅ RAG Fusion complete: {len(final_docs)} unique documents retrieved")
            return final_docs
            
        except Exception as e:
            print(f"❌ RAG Fusion failed: {str(e)}")
            # Fallback to single query retrieval
            print("🔄 Falling back to single query retrieval...")
            return await self.base_retriever.retrieve_documents(query, top_k)
    
    async def _apply_rrf(self, documents: List[dict], top_k: int) -> List[dict]:
        """Apply Reciprocal Rank Fusion to combine document scores"""
        # Group documents by document ID
        doc_groups = defaultdict(list)
        for doc in documents:
            doc_id = doc.get('doc_id', doc.get('metadata', {}).get('document_id', ''))
            if doc_id:
                doc_groups[doc_id].append(doc)
        
        # Calculate RRF scores
        rrf_scores = {}
        for doc_id, doc_list in doc_groups.items():
            rrf_score = 0.0
            
            for doc in doc_list:
                rank = doc.get('rank', 1)
                rrf_score += 1.0 / (self.k + rank)
            
            # Store best document with RRF score
            best_doc = max(doc_list, key=lambda x: x.get('similarity_score', 0))
            best_doc['rrf_score'] = rrf_score
            best_doc['appearance_count'] = len(doc_list)
            best_doc['query_sources'] = [d.get('query_source', 0) for d in doc_list]
            
            rrf_scores[doc_id] = best_doc
        
        # Sort by RRF score
        sorted_docs = sorted(rrf_scores.values(), key=lambda x: x['rrf_score'], reverse=True)
        return sorted_docs[:top_k * 2]  # Return more for deduplication
    
    async def _deduplicate_and_format(self, documents: List[dict], top_k: int) -> List[dict]:
        """Remove duplicates and format final results"""
        seen_content = set()
        final_docs = []
        
        for doc in documents:
            # Create content hash for deduplication
            content_preview = doc.get('content', '')[:100]
            content_hash = hash(content_preview)
            
            if content_hash not in seen_content:
                seen_content.add(content_hash)
                
                # Format final document
                final_doc = {
                    'content': doc.get('content', ''),
                    'similarity_score': doc.get('rrf_score', doc.get('similarity_score', 0.0)),
                    'metadata': doc.get('metadata', {}),
                    'rank': len(final_docs) + 1,
                    'doc_id': doc.get('doc_id', ''),
                    'fusion_metadata': {
                        'rrf_score': doc.get('rrf_score', 0.0),
                        'appearance_count': doc.get('appearance_count', 1),
                        'query_sources': doc.get('query_sources', []),
                        'fusion_method': 'RRF'
                    }
                }
                
                final_docs.append(final_doc)
                
                if len(final_docs) >= top_k:
                    break
        
        return final_docs

class RAGController:
    """Orchestrates the complete RAG pipeline with fusion support"""
    
    def __init__(self, embedder: EmbedderService, retriever: RetrieverService, generator: GeneratorService, fusion_retriever: FusionRetrieverService = None):
        self.embedder = embedder
        self.retriever = retriever
        self.generator = generator
        self.fusion_retriever = fusion_retriever
        self.max_context_length = 2000
    
    async def process_query(self, question: str, history: List[dict] = None, use_fusion: bool = True) -> dict:
        """Complete RAG pipeline: retrieve context and generate response"""
        try:
            print(f"🔍 Processing query: {question}")
            
            # Step 1: Retrieve relevant documents
            if use_fusion and self.fusion_retriever:
                retrieved_docs = await self.fusion_retriever.retrieve_documents_fusion(question)
            else:
                retrieved_docs = await self.retriever.retrieve_documents(question)
            
            print(f"📊 Found {len(retrieved_docs)} similar documents")
            
            # Step 2: Build context from documents
            context_parts = []
            total_chars = 0
            
            for doc in retrieved_docs:
                if total_chars + len(doc['content']) <= self.max_context_length:
                    context_parts.append(f"Source: {doc['metadata']['source']}\n{doc['content']}")
                    total_chars += len(doc['content'])
                else:
                    break
            
            context = "\n\n".join(context_parts)
            print(f"📄 Using {len(context_parts)} documents for context ({len(context)} chars)")
            
            # Step 3: Create prompt with history
            if history:
                history_context = "\n".join([f"Human: {h.get('question', '')}\nAssistant: {h.get('answer', '')}" for h in history[-3:]])
                prompt = f"""Previous conversation:
{history_context}

Context:
{context}

Current question: {question}

Answer based on the context and conversation history:"""
            else:
                prompt = f"""You are a helpful medical assistant. Use the following context to answer the question.

Context:
{context}

Question: {question}

Answer (based only on the context):"""
            
            # Step 4: Generate response
            print(f"🤖 Generating response using {self.generator.model_name}...")
            response = await self.generator.generate_response(prompt, 150)
            
            # Step 5: Format result
            result = {
                'query': question,
                'response': response,
                'sources': [
                    {
                        'title': doc['metadata'].get('title', 'Medical Document'),
                        'source': doc['metadata']['source'],
                        'topic': doc['metadata']['topic'],
                        'similarity': f"{doc['similarity_score']:.3f}",
                        'rank': doc['rank'],
                        'content_preview': doc['content'][:150] + "..."
                    }
                    for doc in retrieved_docs
                ],
                'metadata': {
                    'documentsUsed': len(context_parts),
                    'totalFound': len(retrieved_docs),
                    'contextLength': len(context),
                    'model': self.generator.model_name,
                    'processingTime': datetime.now().isoformat(),
                    'fusionUsed': use_fusion and self.fusion_retriever is not None
                }
            }
            
            return result
            
        except Exception as e:
            print(f"❌ RAGController error: {str(e)}")
            raise

print("✅ FusionRetrieverService and RAGController ready!")

In [ ]:
# Test Supabase connection and RPC functions
print("🧪 Testing Supabase connection...")
try:
    # Test basic connection
    test_result = supabase.table('medical_documents').select('count').execute()
    doc_count = len(test_result.data) if test_result.data else 0
    print(f"✅ Supabase connected - Found {doc_count} documents in database")
    
    # Test RPC function availability
    print("🧪 Testing RPC functions...")
    try:
        stats_result = supabase.rpc('get_document_stats').execute()
        if stats_result.data:
            print("✅ RPC functions working")
            for stat in stats_result.data[:3]:  # Show first 3 document sources
                print(f"   📄 {stat['source']}: {stat['count']} documents")
        else:
            print("⚠️ RPC function exists but returned no data")
    except Exception as rpc_error:
        print(f"⚠️ RPC function test failed: {str(rpc_error)}")
        print("   Vector search will use fallback method")
        
except Exception as e:
    error_str = str(e)
    print(f"⚠️ Supabase connection test failed: {error_str}")
    
    # Provide specific guidance based on error type
    if '401' in error_str or 'Invalid API key' in error_str:
        print("🔧 AUTHENTICATION ERROR - Invalid API Key:")
        print("   ❌ You're using the wrong API key!")
        print("   📋 To fix this:")
        print("   1. Go to your Supabase project dashboard")
        print("   2. Settings → API")
        print("   3. Copy the 'service_role' key (NOT anon key)")
        print("   4. The service_role key is much longer and starts with 'eyJ'")
        print("   5. Re-run Cell 3 with the correct key")
        print("")
        print("   🔍 Key differences:")
        print("   • anon key: Used for client-side apps (WRONG for this notebook)")
        print("   • service_role key: Used for server-side/admin access (CORRECT)")
    elif '404' in error_str:
        print("🔧 TABLE NOT FOUND:")
        print("   ❌ The 'medical_documents' table doesn't exist!")
        print("   📋 To fix this:")
        print("   1. Run the schema.sql in your Supabase SQL editor")
        print("   2. Or run the embed_documents.py script to create tables")
    elif 'timeout' in error_str.lower():
        print("🔧 CONNECTION TIMEOUT:")
        print("   ❌ Can't reach Supabase servers")
        print("   📋 Check your internet connection and Supabase URL")
    else:
        print("🔧 GENERAL CONNECTION ERROR:")
        print("   📋 Common fixes:")
        print("   • Double-check your Supabase URL")
        print("   • Verify you're using service_role key (not anon)")
        print("   • Check if your project is paused/suspended")
        print("   • Ensure database tables exist")
    
    print("\n   ⚠️ The system will continue but may have limited document retrieval")


In [ ]:
# Enhanced FastAPI Setup with Chat History Support
app = FastAPI(
    title="WellnessGrid RAG API",
    description="Medical AI Assistant with RAG Fusion capabilities using FastAPI",
    version="2.0.0",
    docs_url="/docs",
    redoc_url="/redoc"
)

# Add CORS middleware
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],  # Configure appropriately for production
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# Store chat sessions in memory (in production, use Redis or database)
chat_sessions = {}

@app.post("/retrieve-fusion", response_model=RetrieveResponse)
async def retrieve_with_fusion(request: RetrieveRequest):
    """Retrieve documents using RAG Fusion"""
    try:
        print(f"🔄 RAG Fusion retrieval for: {request.query[:100]}...")
        
        # Use fusion retriever
        documents = await fusion_retriever.retrieve_documents_fusion(
            request.query, 
            request.top_k or 5
        )
        
        return RetrieveResponse(
            query=request.query,
            documents=documents,
            total_found=len(documents),
            fusion_metadata={
                "method": "RRF",
                "query_variations": len(set(doc.get('fusion_metadata', {}).get('query_sources', []) for doc in documents)),
                "avg_appearance_count": sum(doc.get('fusion_metadata', {}).get('appearance_count', 1) for doc in documents) / len(documents) if documents else 0
            }
        )
        
    except Exception as e:
        logger.error(f"❌ Fusion retrieval error: {str(e)}")
        raise HTTPException(status_code=500, detail=f"Fusion retrieval failed: {str(e)}")

@app.post("/embed", response_model=EmbedResponse)
async def generate_embedding(request: EmbedRequest):
    """Generate embeddings for text with enhanced error handling"""
    try:
        logger.info(f"🔍 Generating embedding for text: {request.text[:100]}...")
        
        # Use the embedder service
        embedding = await embedder_service.embed_text(request.text)
        
        return EmbedResponse(
            embedding=embedding,
            dimensions=len(embedding),
            model="PubMedBERT (CPU)",
            device="cpu"
        )
        
    except Exception as e:
        logger.error(f"❌ Error in embed endpoint: {str(e)}")
        raise HTTPException(status_code=500, detail=f"Embedding generation failed: {str(e)}")

@app.post("/generate", response_model=GenerateResponse)
async def generate_text(request: GenerateRequest):
    """Enhanced generate endpoint using modular generator service"""
    try:
        logger.info(f"🔬 Generating response for query: {request.query[:100]}...")
        
        # Create enhanced prompt with chat history
        if request.history:
            history_context = "\n".join([f"Human: {h.get('question', '')}\nAssistant: {h.get('answer', '')}" for h in request.history[-3:]])
            prompt = f"""Previous conversation:
{history_context}

Context:
{request.context}

Current question: {request.query}

Answer based on the context and conversation history:"""
        else:
            prompt = f"""You are a helpful medical assistant. Use the following context to answer the question.

Context:
{request.context}

Question: {request.query}

Answer (based only on the context):"""
        
        # Generate response using generator service
        response = await generator_service.generate_response(prompt, request.max_tokens)
        
        return GenerateResponse(
            answer=response,
            model=MODEL_INFO['name'],
            model_path=MODEL_INFO['path'],
            context_used=len(request.context) > 0,
            history_used=len(request.history) > 0
        )
        
    except Exception as e:
        logger.error(f"❌ Error in generate endpoint: {str(e)}")
        raise HTTPException(status_code=500, detail=f"Text generation failed: {str(e)}")

@app.post("/query", response_model=QueryResponse)
async def query_docs(request: QueryRequest):
    """Enhanced query endpoint using modular retriever service"""
    try:
        # Use base retriever service for single query
        results = await base_retriever_service.retrieve_documents(request.query, request.top_k)
        
        return QueryResponse(
            documents=results,
            total_found=len(results),
            query=request.query
        )
        
    except Exception as e:
        logger.error(f"❌ Error in query endpoint: {str(e)}")
        raise HTTPException(status_code=500, detail=f"Document query failed: {str(e)}")

@app.post("/ask", response_model=AskResponse)
async def ask_question(request: AskRequest):
    """Enhanced RAG pipeline with fusion using RAGController"""
    try:
        print(f"🤖 Processing question: {request.question[:100]}...")
        
        # Use the RAG controller with fusion
        result = await rag_controller.process_query(
            question=request.question,
            history=request.history,
            use_fusion=FUSION_CONFIG["enabled"]
        )
        
        # Update chat history
        session_id = request.session_id or "default"
        if session_id not in chat_sessions:
            chat_sessions[session_id] = []
        
        chat_sessions[session_id].append({
            "question": request.question,
            "answer": result['response'],
            "timestamp": datetime.now().isoformat()
        })
        
        # Format response for WellnessGrid compatibility
        return AskResponse(
            response=result['response'],
            sources=result['sources'],
            chat_history=chat_sessions[session_id],
            session_id=session_id,
            mockMode=False,
            metadata=result['metadata']
        )
        
    except Exception as e:
        logger.error(f"❌ Ask endpoint error: {str(e)}")
        raise HTTPException(status_code=500, detail=f"Question processing failed: {str(e)}")

@app.get("/chat/history/{session_id}")
async def get_chat_history(session_id: str):
    """Get chat history for a session"""
    try:
        history = chat_sessions.get(session_id, [])
        return {
            "session_id": session_id,
            "history": history,
            "message_count": len(history)
        }
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

@app.post("/chat/clear/{session_id}")
async def clear_chat_history(session_id: str):
    """Clear chat history for a session"""
    try:
        if session_id in chat_sessions:
            del chat_sessions[session_id]
        
        return {
            "session_id": session_id,
            "cleared": True
        }
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

@app.get("/health", response_model=HealthResponse)
async def health_check():
    """Enhanced health check with memory monitoring"""
    try:
        # Test Supabase connection
        test_result = supabase.table('medical_documents').select('count').execute()
        doc_count = len(test_result.data) if test_result.data else 0
        
        # Test embeddings
        embed_result = supabase.table('document_embeddings').select('count').execute()
        embed_count = len(embed_result.data) if embed_result.data else 0
        
        # Get GPU memory info
        gpu_info = {"available": torch.cuda.is_available()}
        if torch.cuda.is_available():
            allocated = torch.cuda.memory_allocated() / 1024**3
            total = torch.cuda.get_device_properties(0).total_memory / 1024**3
            gpu_info.update({
                "allocated_gb": f"{allocated:.2f}",
                "total_gb": f"{total:.2f}",
                "usage_percent": f"{(allocated/total)*100:.1f}%"
            })
        
        return HealthResponse(
            status="healthy",
            model=MODEL_INFO['name'],
            model_path=MODEL_INFO['path'],
            embedding_device="cpu",
            database="Supabase + pgvector",
            documents_in_db=doc_count,
            embeddings_in_db=embed_count,
            rag_system="fusion_enhanced",
            chat_support=True,
            active_sessions=len(chat_sessions),
            gpu_memory=gpu_info
        )
    except Exception as e:
        raise HTTPException(status_code=503, detail=str(e))

@app.get("/status")
async def status():
    """Detailed status endpoint"""
    try:
        return {
            "timestamp": datetime.now().isoformat(),
            "models": {
                "selected_model": MODEL_INFO['name'],
                "model_loaded": True,
                "embedding_model": "PubMedBERT"
            },
            "database": {
                "connected": True,
                "url": supabase_url[:30] + "..." if supabase_url else "not_set"
            },
            "rag_fusion": {
                "enabled": FUSION_CONFIG["enabled"],
                "query_variations": FUSION_CONFIG["max_query_variations"],
                "rrf_k": FUSION_CONFIG["rrf_k_parameter"]
            },
            "config": CONFIG,
            "memory": {
                "active_sessions": len(chat_sessions),
                "session_ids": list(chat_sessions.keys())
            }
        }
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

print("🌐 Enhanced FastAPI endpoints configured:")
print("  ✅ POST /embed - Generate embeddings")
print(f"  ✅ POST /generate - Generate text with {MODEL_INFO['name']}")
print("  ✅ POST /ask - Enhanced RAG endpoint with RAG Fusion")
print("  ✅ POST /retrieve-fusion - RAG Fusion document retrieval")
print("  ✅ POST /query - Single query document retrieval")
print("  ✅ GET /health - Health check with fusion status")
print("  ✅ GET /status - Detailed status with fusion configuration")
print("  ✅ GET /chat/history/{session_id} - Get chat history")
print("  ✅ POST /chat/clear/{session_id} - Clear chat history")
print("  ✅ GET /docs - Auto-generated API documentation")
print(f"✅ Enhanced FastAPI server ready with RAG Fusion support!")

In [ ]:
# Test RAG Fusion
print("🧪 Testing RAG Fusion...")

test_queries = [
    "What are the symptoms of diabetes?",
    "How to treat high blood pressure?",
    "What causes heart attacks?",
    "Symptoms of stroke"
]

for query in test_queries:
    print(f"\n🔍 Testing: {query}")
    
    # Test single retrieval
    single_docs = await base_retriever_service.retrieve_documents(query, 5)
    print(f"   Single retrieval: {len(single_docs)} docs")
    
    # Test fusion retrieval
    fusion_docs = await fusion_retriever.retrieve_documents_fusion(query, 5)
    print(f"   Fusion retrieval: {len(fusion_docs)} docs")
    
    # Compare results
    single_sources = set(doc.get('metadata', {}).get('source', '') for doc in single_docs)
    fusion_sources = set(doc.get('metadata', {}).get('source', '') for doc in fusion_docs)
    
    new_sources = fusion_sources - single_sources
    print(f"   New sources found: {len(new_sources)}")
    
    if new_sources:
        print(f"   New sources: {list(new_sources)[:3]}")
    
    # Test full RAG pipeline
    try:
        rag_result = await rag_controller.process_query(query, use_fusion=True)
        print(f"   RAG response: {rag_result['response'][:100]}...")
        print(f"   Sources used: {rag_result['metadata']['documentsUsed']}")
        print(f"   Fusion enabled: {rag_result['metadata']['fusionUsed']}")
    except Exception as e:
        print(f"   ❌ RAG test failed: {str(e)}")

print("\n✅ RAG Fusion testing complete!")
print("🎯 System is ready for deployment with enhanced retrieval capabilities")

In [ ]:
# GPU Memory Management Utilities
def get_gpu_memory_info():
    """Get detailed GPU memory information"""
    if not torch.cuda.is_available():
        return "No GPU available"
    
    device = torch.cuda.current_device()
    allocated = torch.cuda.memory_allocated(device) / 1024**3
    reserved = torch.cuda.memory_reserved(device) / 1024**3
    total = torch.cuda.get_device_properties(device).total_memory / 1024**3
    free = total - allocated
    
    return {
        "allocated": f"{allocated:.2f} GB",
        "reserved": f"{reserved:.2f} GB", 
        "total": f"{total:.2f} GB",
        "free": f"{free:.2f} GB",
        "percentage_used": f"{(allocated/total)*100:.1f}%"
    }

def clear_gpu_memory():
    """Clear GPU memory cache"""
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        print("🧹 GPU cache cleared")
    else:
        print("📱 No GPU to clear")

# Show current GPU memory status
print("🔍 Current GPU Memory Status:")
memory_info = get_gpu_memory_info()
if isinstance(memory_info, dict):
    for key, value in memory_info.items():
        print(f"   {key}: {value}")
else:
    print(f"   {memory_info}")

# Add GPU memory management to Flask health endpoint
print("✅ GPU memory utilities ready!")
print("   Use get_gpu_memory_info() to check memory")
print("   Use clear_gpu_memory() to free cache")


In [ ]:
# Server startup with ngrok (optional for local development)
from pyngrok import ngrok
import uvicorn
import nest_asyncio

# Quick pre-check to give immediate feedback
print("🔍 Pre-flight check...")
required_vars = ['medical_model', 'tokenizer', 'supabase', 'rag_controller', 'CONFIG', 'MODEL_INFO']
missing_vars = [var for var in globals() if var not in globals()]

# Check if we have the essential components
essential_components = ['MODEL_INFO', 'supabase', 'rag_controller']
missing_essential = [comp for comp in essential_components if comp not in globals()]

if missing_essential:
    print(f"❌ Missing essential components: {', '.join(missing_essential)}")
    print(f"🔧 Please run all previous cells in order first!")
    print(f"   Then come back to this cell.")
    raise RuntimeError(f"Required setup incomplete. Missing: {', '.join(missing_essential)}")

print("✅ Pre-flight check passed!")
print(f"🎯 Selected model: {MODEL_INFO['name']}")

# Setup ngrok if token is available
if ngrok_token and COLAB_AVAILABLE:
    print("🔑 Using ngrok auth token from Colab secrets...")
    ngrok.set_auth_token(ngrok_token)
    print("✅ Ngrok token set successfully!")
    
    # Start ngrok tunnel
    print("🌐 Starting ngrok tunnel...")
    public_url = ngrok.connect(8000)
    print(f"🌍 Public URL: {public_url}")
    print("? Copy this URL to your WellnessGrid app configuration!")
    
elif ngrok_token and not COLAB_AVAILABLE:
    print("🔑 Using ngrok auth token from environment...")
    ngrok.set_auth_token(ngrok_token)
    print("✅ Ngrok token set successfully!")
    
    # Start ngrok tunnel
    print("🌐 Starting ngrok tunnel...")
    public_url = ngrok.connect(8000)
    print(f"🌍 Public URL: {public_url}")
    print("📋 Copy this URL to your WellnessGrid app configuration!")
    
else:
    print("📱 No ngrok token found - server will run locally only")
    print("🌍 Local URL: http://localhost:8000")
    print("💡 Set NGROK_AUTH_TOKEN environment variable to enable public access")

# Start FastAPI app with uvicorn
print("🚀 Starting FastAPI app with uvicorn...")
print("📡 Available endpoints:")
print("  ✅ POST /embed - Generate embeddings (required by WellnessGrid)")
print(f"  ✅ POST /generate - Generate text with {MODEL_INFO['name']} (required by WellnessGrid)")
print("  ✅ POST /ask - Main RAG endpoint for WellnessGrid with RAG Fusion")
print("  ✅ POST /retrieve-fusion - RAG Fusion document retrieval")
print("  ✅ GET /health - Health check")
print("  ✅ POST /query - Query documents from Supabase")
print("  ✅ GET /docs - Interactive API documentation")
print("  ✅ GET /redoc - Alternative API documentation")

if 'public_url' in locals():
    print(f"\n🎯 IMPORTANT: Copy the ngrok URL above to your WellnessGrid .env.local:")
    print(f"   FAST_API_URL={public_url}")
else:
    print(f"\n🎯 For local development, use:")
    print(f"   FAST_API_URL=http://localhost:8000")

print("\n⚠️  Keep this cell running to maintain the server!")
print(f"\n🚀 Your WellnessGrid RAG system with {MODEL_INFO['name']} and RAG Fusion is now live!")

# Apply nest_asyncio to allow nested event loops in Jupyter
nest_asyncio.apply()

# Run the server
try:
    uvicorn.run(app, host="0.0.0.0", port=8000, log_level="info")
except KeyboardInterrupt:
    print("\n⏹️ Server stopped by user")
except Exception as e:
    print(f"\n❌ Server error: {str(e)}")
    print("🔧 Try restarting the notebook if you encounter issues")